<a href="https://colab.research.google.com/github/prxrwx/Simulation/blob/main/Example-queue-simulate3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import simpy
import random

SimPy in Queuing simulation for non-linear

In [ ]:
# Arrivals generator function
def patient_generator_ed(env, ed_inter, mean_regis, mean_triage,
                         mean_ed_assess, mean_acu_assess,
                         receptionist, nurse, ed_doctor, acu_doctor):
  p_id = 0

  while True:
    # Create new instance of activity_generator
    p = activity_generator_ed(env, mean_regis, mean_triage, 
                              mean_ed_assess, mean_acu_assess,
                              receptionist, nurse, ed_doctor, acu_doctor, p_id)
    # Run the activity generator
    env.process(p)

    # Sample time to next arrival
    t = random.expovariate(1.0/ed_inter)
    
    # Freeze until time elapsed
    yield env.timeout(t)

    p_id += 1

In [ ]:
# Activity generator function
def activity_generator_ed(env, mean_regis, mean_triage, 
                          mean_ed_assess, mean_acu_assess,
                          receptionist, nurse, ed_doctor, acu_doctor, p_id):
  time_enqueue_of_registration = env.now

  # Request a receptionist
  with receptionist.request() as req:
    # Freeze until the request can be met
    yield req

    time_dequeue_of_registration = env.now
    time_in_queue_for_registration = (time_dequeue_of_registration - 
                                      time_enqueue_of_registration)
    
    print("Patient ",p_id, " queued for registration for ", 
          time_in_queue_for_registration, " minutes.",sep="")
    # Sample the time spent in registration
    sampled_registration_time = random.expovariate(1.0/mean_regis)

    # Freeze until that time has elapsed
    yield env.timeout(sampled_registration_time)

# The above with statement, assign with the receptionist. 
# Now start for queuing of the nurse.
  time_enqueue_of_nurse = env.now

  # Request a nurse
  with nurse.request() as req:
    # Freeze until the request can be met
    yield req

    # Calculate time patient was queuing
    time_dequeue_of_nurse = env.now
    time_in_queue_for_nurse = (time_dequeue_of_nurse -
                               time_enqueue_of_nurse)
    print("Patient " , p_id, " queued for triage for ", 
          time_in_queue_for_nurse, " minutes.", sep="")
    # Sample time spent with nurse
    sampled_triage_time = random.expovariate(1.0/mean_triage)

    # Freeze until that time has passed
    yield env.timeout(sampled_triage_time)     

    # Branching path for ACU or ED 
    # We split them out using percentages probabilities, so let's randomly
    # sample from a uniform distribution to decide ACU or ED          
    decide_acu_branch = random.uniform(0,1)             
    if decide_acu_branch < 0.2:
      time_enqueue_of_acu_assess = env.now

      # Request an ACU doctor
      with acu_doctor.request() as req:
        # Freeze until the request can be met
        yield req

        time_dequeue_of_acu_assess = env.now
        time_in_queue_for_acu_assess = (time_dequeue_of_acu_assess -
                                        time_enqueue_of_acu_assess)
        print("Patient ",p_id, " queued for ACU assessment for ",
              time_in_queue_for_acu_assess, " minutes.", sep="")
        # Sample the time spend being assessed by the ACU doctor
        sampled_acu_assess_time = random.expovariate(1.0/mean_acu_assess)
        # Freeze until that time has elapsed
        yield env.timeout(sampled_acu_assess_time)
    else:
      time_enqueue_of_ed_assess = env.now
      # Request an ED doctor
      with ed_doctor.request() as req:
        # Freeze until the request can be met
        yield req
        
        time_dequeue_of_ed_assess = env.now
        time_in_queue_for_ed_assess = (time_dequeue_of_ed_assess -
                                       time_enqueue_of_ed_assess)
        print("Patient ",p_id, " queued for ED assessment for ",
              time_in_queue_for_ed_assess, " minutes.", sep="")

In [ ]:
# Set up simulation environment
env = simpy.Environment()

# Set up resources
receptionist = simpy.Resource(env, capacity=1)
nurse = simpy.Resource(env, capacity=2)
ed_doctor = simpy.Resource(env, capacity=2)
acu_doctor = simpy.Resource(env, capacity=1)

# Set up parameter values
ed_inter = 8
mean_regis = 2
mean_triage = 5
mean_ed_assess = 30
mean_acu_assess = 60

# Start the arrivals generator
env.process(patient_generator_ed(env, ed_inter, mean_regis, mean_triage,
                                 mean_ed_assess, mean_acu_assess, 
                                 receptionist, nurse, ed_doctor, acu_doctor))

# Run the simulation
env.run(until=400)

Patient 0 queued for registration for 0 minutes.
Patient 0 queued for triage for 0.0 minutes.
Patient 1 queued for registration for 2.796356302718619 minutes.
Patient 1 queued for triage for 0.0 minutes.
Patient 2 queued for registration for 1.9388538069013728 minutes.
Patient 0 queued for ED assessment for 0.0 minutes.
Patient 2 queued for triage for 0.0 minutes.
Patient 3 queued for registration for 0.8307287666889005 minutes.
Patient 1 queued for ACU assessment for 0.0 minutes.
Patient 2 queued for ED assessment for 0.0 minutes.
Patient 3 queued for triage for 4.645954561701673 minutes.
Patient 4 queued for registration for 0.0 minutes.
Patient 3 queued for ED assessment for 0.0 minutes.
Patient 4 queued for triage for 2.7034159953343 minutes.
Patient 4 queued for ED assessment for 0.0 minutes.
Patient 5 queued for registration for 0.0 minutes.
Patient 5 queued for triage for 0.0 minutes.
Patient 6 queued for registration for 0.0 minutes.
Patient 5 queued for ED assessment for 0.0 m